In [3]:
from itertools import cycle
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
import re
import string

In [2]:
def read_input(day):
    with open(f'Inputs/input{day}') as f:
        return f.read().rstrip('\n')

#Norvig functions
def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))

def Array(lines):
    "Parse an iterable of str lines into a 2-D array. If `lines` is a str, splitlines."
    if isinstance(lines, str): lines = lines.splitlines()
    return mapt(Vector, lines)

def Vector(line):
    "Parse a str into a tuple of atoms (numbers or str tokens)."
    return mapt(Atom, line.replace(',', ' ').split())

def Integers(text): 
    "Return a tuple of all integers in a string."
    return mapt(int, re.findall(r'-?\b\d+\b', text))

def Integers2(text): 
    "Return a tuple of all integers in a string."
    return mapt(int, re.findall(r'\d+', text))

def Atom(token):
    "Parse a str token into a number, or leave it as a str."
    try:
        return int(token)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return token

# Day 1

### Part 1

In [8]:
changes = [int(s) for s in read_input(1).split()]
changes[-5:]

[9, 1, -11, 8, -121858]

In [9]:
sum(changes)

423

### Part 2

In [16]:
def find_repeated_freq(changes):
    observed_freqs = set()
    current_freq = 0
    for change in cycle(changes):
        observed_freqs.add(current_freq)
        current_freq += change
        if current_freq in observed_freqs:
            return current_freq        

In [17]:
find_repeated_freq(changes)

61126

# Day 2

### Part 1

In [3]:
box_ids = read_input(2).split()
box_ids[-5:]

['bpacnmethizqynfajoxtvkwudr',
 'bpocnmclhizqygfsjoxtvkwukr',
 'zpacnmwlhizqygfsjoxzvkwudr',
 'bpacpoelhqzqygfsjoxtvkwudr',
 'bpacnlelhizqyzfsjoxtvkwukr']

In [5]:
letter_counts = [Counter(x) for x in box_ids]

In [7]:
sum([2 in count.values() for count in letter_counts]) * sum([3 in count.values() for count in letter_counts])

8820

### Part 2

In [17]:
def edit_list(s):
    return [s[:i] + '*' + s[i+1:] for i in range(len(s))]

In [18]:
edit_list('abcde')

['*bcde', 'a*cde', 'ab*de', 'abc*e', 'abcd*']

In [20]:
seen_edits = set()
for box_id in box_ids:
    edits = edit_list(box_id)
    for edit in edits:
        if edit in seen_edits:
            print(edit.replace('*', ''))
        else:
            seen_edits.add(edit)

bpacnmglhizqygfsjixtkwudr


# Day 3

### Part 1

In [5]:
claims = read_input(3).splitlines()
claims[-5:]

['#1229 @ 488,649: 27x21',
 '#1230 @ 28,723: 27x15',
 '#1231 @ 920,956: 18x23',
 '#1232 @ 23,641: 17x13',
 '#1233 @ 437,811: 19x10']

In [12]:
Integers2(claims[-1])

(1233, 437, 811, 19, 10)

In [20]:
def parse_claim(claim):
    claim_id, hbuffer, vbuffer, hlen, vlen = Integers2(claim)
    hslice = slice(hbuffer, hbuffer + hlen)
    vslice = slice(vbuffer, vbuffer + vlen)
    return claim_id, hslice, vslice

assert(parse_claim('#123 @ 3,2: 5x4') == (123, slice(3, 8), slice(2, 6)))

In [21]:
fabric = np.zeros([1000, 1000])
for claim in claims:
    _, hslice, vslice = parse_claim(claim)
    fabric[hslice, vslice] += 1
(fabric > 1).sum()

98005

### Part 2

In [22]:
for claim in claims:
    claim_id, hslice, vslice = parse_claim(claim)
    if (fabric[hslice, vslice] == 1).all():
        print(claim_id)

331


# Day 4

### Part 1

In [6]:
records = sorted(read_input(4).split('\n'))
records[:10]

['[1518-02-12 23:50] Guard #1789 begins shift',
 '[1518-02-13 00:05] falls asleep',
 '[1518-02-13 00:25] wakes up',
 '[1518-02-13 00:40] falls asleep',
 '[1518-02-13 00:52] wakes up',
 '[1518-02-14 00:01] Guard #2617 begins shift',
 '[1518-02-14 00:22] falls asleep',
 '[1518-02-14 00:29] wakes up',
 '[1518-02-14 00:47] falls asleep',
 '[1518-02-14 00:50] wakes up']

We need to be careful, since a guard can start their shift on the previous day.

In [55]:
def parse_records(records):
    IDs = []
    all_times = []
    for i, record in enumerate(records):
        record = record.split()
        if len(record) == 6:
            IDs.append(record[3])
            if i != 0:
                all_times.append(times)
            times = []
        else:
            times.append(int(record[1][3:5]))
    all_times.append(times)
    return IDs, all_times

In [30]:
def get_date(record):
    date = record.split()[0].strip('[')
    return date.replace('5', '9', 1) #change the year for pandas

assert(get_date('[1518-02-13 00:05] falls asleep') == '1918-02-13')

In [56]:
IDs, all_times = parse_records(records)
date = pd.date_range(start=get_date(records[1]), end=get_date(records[-1]))

In [50]:
def parse_times(all_times):
    minute_array = np.zeros([len(all_times), 60])  
    for row, times in enumerate(all_times):
        for i in range(0, len(times), 2):
            minute_array[row, times[i]:times[i+1]] = 1
    return minute_array                

In [57]:
minute = parse_times(all_times)

In [53]:
all_times[:5]

[[5, 25, 40, 52],
 [22, 29, 47, 50, 53, 56],
 [1, 25, 35, 45],
 [21, 47],
 [21, 45, 54, 58]]

In [106]:
minute[:1]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [98]:
df = pd.DataFrame(data=minute, index=date)
df['ID'] = IDs

In [99]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,ID
1918-02-13,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,#1789
1918-02-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,#2617
1918-02-15,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,#1997
1918-02-16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,#3041
1918-02-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,#1873


In [100]:
id_rank = df.groupby('ID').sum().sum(axis=1).sort_values(ascending=False)
id_rank.head()

ID
#3041    542.0
#1901    502.0
#1997    498.0
#1069    478.0
#1789    443.0
dtype: float64

In [101]:
guard = id_rank.index[0]
minute_rank = df.groupby('ID').sum().loc[guard].sort_values(ascending=False)
minute_rank.head()

39    16.0
42    15.0
36    15.0
38    15.0
43    15.0
Name: #3041, dtype: float64

In [102]:
int(guard[1:]) * minute_rank.index[0]

118599

### Part 2

In [103]:
id_rank = df.groupby('ID').sum().max(axis=1).sort_values(ascending=False)
id_rank.head()

ID
#1997    17.0
#3041    16.0
#1901    15.0
#2213    14.0
#1069    13.0
dtype: float64

In [104]:
guard = id_rank.index[0]
minute_rank = df.groupby('ID').sum().loc[guard].sort_values(ascending=False)
minute_rank.head()

17    17.0
18    16.0
19    15.0
16    15.0
20    14.0
Name: #1997, dtype: float64

In [105]:
int(guard[1:]) * minute_rank.index[0]

33949

Well, I would have solved that differently if I had realised that the date was a red herring :(
That is the disadvantage with part 2 being unknown - you have to make guesses about the best data structures.

# Day 5

### Part 1

I tried a number of approaches to solving this. Slowest to fastest:
- re.sub(pattern='aA|Aa|bB...', replace='')
- iterate through string, and remove reactions by concatenating the two sides (the concat is expensive)
- iterate through list of chars, and remove reactions by pop() (pop is O(n) if not from the end of a list)
- current solution, since it makes the pop() efficient!

In [64]:
def trigger_all(polymer):
    polymer = list(reversed(polymer))
    final_polymer = []
    while len(polymer):
        final_polymer.append(polymer.pop())
        try:
            while polymer[-1] == final_polymer[-1].swapcase():
                polymer.pop()
                final_polymer.pop()
        except IndexError:
            continue
    return ''.join(final_polymer)

assert(trigger_all('') == '')
assert(trigger_all('aA') == '')
assert(trigger_all('aBbA') == '')
assert(trigger_all('abAB') == 'abAB')
assert(trigger_all('dabAcCaCBAcCcaDA') == 'dabCBAcaDA')

In [66]:
len(trigger_all(read_input(5)))

9822

### Part 2

In [67]:
polymer = read_input(5)
lengths = []
for letter in string.ascii_lowercase:
    new_polymer = re.sub(letter, '', polymer, flags=re.IGNORECASE)
    lengths.append(len(trigger_all(new_polymer)))
min(lengths)

5726

# Day 6

In [3]:
def manhattan_distance(a, b):
    return abs(b[0]-a[0]) + abs(b[1]-a[1])
    
assert(manhattan_distance((0,0), (3,4)) == 7)

In [4]:
def closest_coord(x, coords):
    "Returns index of closest coord, or -1 if it's a tie"
    distances = [manhattan_distance(x, c) for c in coords]
    sorted_distances = sorted(distances)
    if sorted_distances[0] == sorted_distances[1]:
        return -1
    else:
        return distances.index(sorted_distances[0])

assert(closest_coord((0,0), ((1,1), (2,2))) == 0)
assert(closest_coord((0,0), ((1,1), (0,0))) == 1)
assert(closest_coord((0,0), ((1,1), (2,0))) == -1)

In [5]:
coords = Array(read_input(6))
coords[:5]

((181, 47), (337, 53), (331, 40), (137, 57), (200, 96))

In [6]:
max_x = max([c[0] for c in coords])
max_y = max([c[1] for c in coords])
grid_dims = (max_x + 1, max_y + 1)

In [7]:
%%prun
grid = np.empty(grid_dims, dtype=np.int8)
for i in range(grid_dims[0]):
    for j in range(grid_dims[1]):
        grid[i, j] = closest_coord((i, j), coords)        

In [8]:
ranking = Counter(list(grid.flatten()))
border_values = {*grid[0, :], *grid[:, 0], *grid[-1, :], *grid[:, -1]}
final_ranking = {k: v for k,v in ranking.items() if k not in border_values}

In [9]:
max(final_ranking.values())

4233

### Part 2

In [10]:
def coords_distance_sum(x, coords):
    "Returns sum of distances to each coord"
    return sum([manhattan_distance(x, c) for c in coords])

assert(coords_distance_sum((0,0), ((1,1), (2,2))) == 6)
assert(coords_distance_sum((0,0), ((1,1), (0,0))) == 2)
assert(coords_distance_sum((0,0), ((1,1), (2,0))) == 4)

In [33]:
%%prun
grid = np.empty(grid_dims, dtype=np.int32)
for i in range(grid_dims[0]):
    for j in range(grid_dims[1]):
        grid[i, j] = coords_distance_sum((i, j), coords) 

In [34]:
(grid < 10000).sum()

45290

In [70]:
def manhattan_grid(c):
    grid = np.empty(grid_dims, dtype=np.int32)
    for i in range(grid_dims[0]):
        grid[i, 0] = manhattan_distance((i,0), c)
    base = grid[:, 0]
    for j in range(1, grid_dims[1]):
        grid[:, j] = base - base.min() + abs(j-c[1])
    return grid  

In [75]:
%%prun
grid = sum([manhattan_grid(c) for c in coords])
(grid < 10000).sum()

45290

# Day 7

### Part 1

In [4]:
test_instructions = ['Step C must be finished before step A can begin',
                     'Step C must be finished before step F can begin',
                     'Step A must be finished before step B can begin',
                     'Step A must be finished before step D can begin',
                     'Step B must be finished before step E can begin',
                     'Step D must be finished before step E can begin',
                     'Step F must be finished before step E can begin'
                    ]

In [15]:
def parse_instructions(instructions):
    children = defaultdict(list)
    parents = defaultdict(list)
    steps = set()
    for i in instructions:
        i = i.split()
        parent, child = i[1], i[7]
        parents[child].append(parent)
        children[parent].append(child)
        steps.add(parent)
        steps.add(child)
    return children, parents, steps

In [24]:
def find_possible_starts(parents, steps):
    return {s for s in steps if not parents[s]}

In [25]:
c, p, s = parse_instructions(test_instructions)
assert('B' in c['A'])
assert('D' in c['A'])
assert(c['E'] == [])

assert(find_possible_starts(p, s) == {'C'})

In [26]:
def find_sequence(instructions):
    children, parents, steps = parse_instructions(instructions)
    candidates = find_possible_starts(parents, steps)
    sequence = []
    while candidates:
        for c in sorted(candidates):
            if all([(p in sequence) for p in parents[c]]): #verify all preceding steps completed
                next_step = c
                break
        candidates.remove(next_step)
        sequence.append(next_step)
        candidates.update(children[next_step])
    return ''.join(sequence)

In [27]:
find_sequence(test_instructions)

'CABDFE'

In [28]:
find_sequence(read_input(7).splitlines())

'BETUFNVADWGPLRJOHMXKZQCISY'

### Part 2

In [45]:
def job_cost(c, base_cost):
    return ord(c) - ord('A') + 1 + base_cost

def progress_jobs(assigned):
    for worker in assigned:
        if assigned[worker]:
            assigned[worker][1] -= 1

def find_freed_workers(assigned):
    return [worker for worker in assigned if assigned[worker] and assigned[worker][1] == 0]

In [54]:
def find_cost(instructions, num_workers=5, base_cost=60):
    children, parents, steps = parse_instructions(instructions)
    candidates = find_possible_starts(parents, steps)
    completed = set()
    time = 0
    assigned = {i: None for i in range(num_workers)} #we will assign tasks to workers
    while len(completed) < len(steps):
        #assign jobs
        available = [worker for worker in assigned if not assigned[worker]]
        for c in sorted(candidates):
            if all([(p in completed) for p in parents[c]]) and available: #verify all preceding steps completed
                assigned[available.pop()] = [c, job_cost(c, base_cost)]
                candidates.remove(c)
        #increment time and progress jobs
        time += 1
        progress_jobs(assigned)
        freed = find_freed_workers(assigned)
        for worker in freed:
            job = assigned[worker]
            completed.add(job[0])
            candidates.update(children[job[0]])
            assigned[worker] = None
    return time

In [62]:
find_cost(test_instructions, num_workers=2, base_cost=0)

15

In [63]:
find_cost(read_input(7).splitlines())

848